In [ ]:
from flask import Flask, render_template, url_for, redirect, request, make_response, Response, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
from flask_cors import CORS
import MySQLdb

app = Flask(__name__)
# CORS(app)

# connection = pymysql.connect(host="localhost", 
connection = MySQLdb.connect(host="localhost",
                     user="root",   
                     passwd="root", 
                     db="love4cocktail",   
                     unix_socket="/Applications/MAMP/tmp/mysql/mysql.sock",
                     charset='utf8')      
#cursorclass=pymysql.cursors.DictCursor)

######### Login Manager Setup ##########

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"


@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id
    
############ Web Page Routes Setup ###############
    
@app.route("/")
@login_required
def home():
    return render_template('home.html', message="You are my cute pig!")

@app.route("/login")
def login():
    message = 'Please login in first.'
    return render_template('login.html', message=message)

@app.route('/query_auth')
def query_auth():
    username = request.args.get("username")
    password = request.args.get("password")
    if username == "vvnwoo@163.com" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to vvn's pages. Your login has been granted."
        return make_response(message, 200, )
    else:
        message = 'Wrong password!'
        return make_response( messagte,401,{'WWW-Authenticate':'Basic realm="Login Required"'})

@app.route("/form_auth",methods=['POST'])
def form_auth():
    username = request.form['email']
    password = request.form['pwd']
    if username == "vvnwoo@163.com" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to vvn's pages. Your login has been granted."
        return render_template('home.html', message=message)
    else:
        message = 'Wrong password!'
        return render_template('login.html',message=message)
    


@app.route("/album")
@login_required
def index():
    user = {"name":"vvnwoo"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM little_menu"
        cursor.execute(sql)
        # Display results
        result = cursor.fetchall()
#     connection.close()
    # print(result)
    # print('\n')
    final_result = [list(i) for i in result]
    dataset=[]
    dictionary={}
    for i in final_result:
        dictionary['cocktail'] = i[0]
        dictionary['flavour_2'] = i[1]
        dictionary['spirit_type'] = i[3]
        dictionary['alcohol_content'] = i[2]
        dictionary['recipe'] = i[6]
        dictionary['url'] = i[7]
        dataset.append(dictionary.copy())
        # print(i)
        # print(dict)
#         dataset.append(dict.copy()) #markers.append(fld.copy())
        dictionary = {}
    
    return render_template('index.html',album_user=user, dataset=dataset)

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = 'Thanks for logging out.'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = 'Wrong password!'
    return render_template('login.html',message=message),500

######### API Endpoints ##########    

@app.route('/api_auth', methods=['POST'])
def api_auth():
    username = request.json['username']
    password = request.json['password']
    if username == "vvnwoo@163.com" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to vvn's pages. Your login has been granted."
        return jsonify({'response':'ok!','message':message})
    else:
        message = 'Wrong password!'
        return jsonify({'response':'Invalid!','message':'Cannot authenticate.'})
    


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 3301, app)
# if __name__ == '__main__':
#        app.run(debug = True)

 

 * Running on http://localhost:3301/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2020 00:19:28] "GET / HTTP/1.1" 200 -
